<a href="https://colab.research.google.com/github/softmurata/colab_notebooks/blob/main/llm/OpenAssistant_Llama2_13B_Orca_v2_8K_3166_GPTQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install auto-gptq

Load model

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

model_name_or_path = "TheBloke/OpenAssistant-Llama2-13B-Orca-v2-8K-3166-GPTQ"
model_basename = "gptq_model-4bit-128g"

use_triton = False

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=False,
        device="cuda:0",
        use_triton=use_triton,
        quantize_config=None)

Inference

In [ ]:
prompt = "AIについて教えて"
prompt_template=f'''<|prompter|>{prompt}<|endoftext|><|assistant|>
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]).split("<|assistant|>")[-1].split("<|endoftext|>")[0])

In [ ]:
prompt_template='''<|prompter|>replace chair into sofa at the left side<|endoftext|>
<|assistant|>{"part": "left", "source object": ["chair"], "target object": ["sofa"]}<|endoftext|>
<|prompter|>change wooden table into white table in the right part<|endoftext|>
<|assistant|>{"part": "right", "source object": ["wooden table"], "target object": ["white table"]}<|endoftext|>
<|prompter|>replace blue chair and red sofa into yellow table and green chair at the bottom<|endoftext|>
<|assistant|>{"part": "bottom", "source object": ["blue chair", "red sofa"], "target object": ["yellow table", "green chair"]}<|endoftext|>
<|prompter|>replace sofa and shelf into chair and picture at the right side in the room<|endoftext|>
<|assistant|>'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]).split("<|assistant|>")[-1].split("<|endoftext|>")[0])

In [ ]:
prompt = """Please summarize the following context at the bullet points

Context:
    Elon Musk is a well-known entrepreneur, inventor, and engineer. He was born on June 28, 1971, in Pretoria, South Africa, and later moved to the United States where he pursued his education and built his career. Musk is the founder and CEO of several successful companies, including SpaceX, Tesla, Neuralink, and The Boring Company.

    SpaceX is a private space exploration company that aims to make space travel affordable and accessible. Tesla is an electric car company that focuses on developing sustainable transportation solutions. Neuralink is a company that works on developing brain-machine interfaces to improve human cognitive abilities. The Boring Company aims to revolutionize tunneling technology to reduce the cost and time required for tunnel construction.

    Elon Musk is known for his ambitious goals, innovative ideas, and ability to disrupt established industries. He has been recognized for his contributions to science and technology and has received numerous awards and honors throughout his career.
"""
prompt_template=f'''<|prompter|>{prompt}<|endoftext|><|assistant|>
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, max_new_tokens=512)
print(tokenizer.decode(output[0]).split("<|assistant|>")[-1].split("<|endoftext|>")[0])